**1. Bài tập về xử lý các trường hợp dữ liệu bị thiếu**

Trong bài tập này, chúng ta sẽ làm việc với bộ dữ liệu Housing Prices. 

Tập dữ liệu này có 79 giá trị mô tả (gần như) mọi khía cạnh của các căn nhà ở tại Ames, Iowa.
Với tập dữ liệu này, chúng ta cần xây dựng mô hình để dự đoán giá của mỗi ngôi nhà.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Đọc dữ liệu
X_full = pd.read_csv('https://raw.githubusercontent.com/ltdaovn/dataset/master/housing-prices/train.csv',
                     index_col='Id')
X_test_full = pd.read_csv('https://raw.githubusercontent.com/ltdaovn/dataset/master/housing-prices/test.csv',
                          index_col='Id')

# Loại bỏ các căn nhà không có giá
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Để đơn giản bài toán, ở đây chúng ta chỉ chọn các thuộc tính số
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Chia tập dữ liệu thành 2 tập dữ liệu con là training set và validation set
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8,
                                                      test_size=0.2,
                                                      random_state=0)

# Xem các căn nhà đầu tiên trong tập dữ liệu huấn luyện. Chú ý các giá trị bị thiếu.
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


**Bước 1: Làm quen với dữ liệu**

In [2]:
# Xem mô tả tập dữ liệu huấn luyện
print(X_train.shape)

# Số lượng dữ liệu bị thiếu trong các cột
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


**Hãy trả lời các câu hỏi sau đây?**

Có bao nhiêu căn nhà trong tập dữ liệu huấn luyện?

num_rows = **1168**

Có bao nhiêu cột dữ liệu bị thiếu?

num_cols_with_missing = **3**

How many missing entries are contained in all of the training data?

tot_missing = **212 + 6 + 58 = 276**

Theo bạn, phương pháp nào thích hợp **nhất** để xử lý trường hợp bị thiếu dữ liệu này?

--> Sử dụng **phương pháp xoá cột**

**Bước 2: định nghĩa hàm để đo chất lượng của từng phương pháp**

Để so sánh chất lượng của các phương pháp, chúng ta cần định nghĩa hàm score_dataset(). 

Hàm được sử dụng trong ví dụ này là hàm Trung bình của sai biệt tuyệt đối (Sai số tuyệt đối trung bình)(the mean absolute error (MAE)) 
dành cho mô hình rừng ngẫu nhiên (RandomForest).
(https://en.wikipedia.org/wiki/Mean_absolute_error) 

In [37]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, accuracy_score

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

**Bước 3:**

Trong các phương pháp xử lý dữ liệu bị thiếu bạn đã học,
phương pháp nào cho kết quả dự báo chính xác nhất.
Hãy cho biết giá trị MAE của từng mô hình.

--> **Phương pháp xoá cột** hiệu quả nhất với MAE = 17837.825708 thấp nhất

**3.a. Phương pháp 1: Xóa cột có dữ liệu bị thiếu**

In [38]:
# Lấy tên các cột có dữ liệu bị thiếu
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Xóa các cột này trong tập training và validation
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

MAE_Score = score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid)
print("MAE from Approach 1 (Drop columns with missing values) = %f" % MAE_Score)

MAE from Approach 1 (Drop columns with missing values) = 17837.825708


**3.b. Phương pháp 2: Thay thế**

Thư viện Scikit-Learn cung cấp một lớp hiệu quả để chúng ta giải quyết các giá trị thiếu:
SimpleImputer.

In [12]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()

imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

# Đánh giá mô hình
MAE_Score = score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid)
print("MAE from Approach 2 (Imputation) = %f" % MAE_Score)

MAE from Approach 2 (Imputation) = 18062.894612


**3.c. Phương pháp 3: cải tiến phương pháp thay thế**

In [13]:
# Để đảm bảo an toàn, tạo bản sao dữ liệu
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Tạo các cột mới để đánh dấu các vị trí dữ liệu được thay thế

for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Phương pháp thay thế cải tiến my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

# Đánh giá mô hình

MAE_Score = score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid)
print("MAE from Approach 3 (An Extension to Imputation) = %f" % MAE_Score)

MAE from Approach 3 (An Extension to Imputation) = 18148.417180
